# 집계값 features로 overSampling

id 별로 groupby하여 집계값을 지정할 경우 데이터의 개수가 3125개로 너무 적음.   
하지만 제출할 submission의 index값이 id로 지정되어 test 역시 grouping 이 불가피한 것으로 생각됨.  
train set을 더 확장함으로서 과적합을 방지하고 모델의 고도화를 기대함.   

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

import time
import datetime
pd.set_option('display.max_columns', None)

train = pd.read_csv('train_features.csv')
test = pd.read_csv('test_features.csv')
train_label = pd.read_csv('train_labels.csv')
submission = pd.read_csv('sample_submission.csv')

In [7]:
train_label.label_desc.value_counts()

Non-Exercise                                    1518
Device on Table                                   97
Plank                                             55
Wall Squat                                        48
Jump Rope                                         47
                                                ... 
Triceps Kickback (knee on bench) (right arm)      13
Arm Band Adjustment                               12
Dumbbell Row (knee on bench) (left arm)           12
Triceps Kickback (knee on bench) (left arm)       12
Dumbbell Row (knee on bench) (right arm)          12
Name: label_desc, Length: 61, dtype: int64

In [9]:
train_label

,id,label,label_desc
0,0,37,Shoulder Press (dumbbell)
1,1,26,Non-Exercise
2,2,3,Biceps Curl (band)
3,3,26,Non-Exercise
4,4,26,Non-Exercise
...,...,...,...
3120,3120,26,Non-Exercise
3121,3121,26,Non-Exercise
3122,3122,15,Dynamic Stretch (at your own pace)
3123,3123,26,Non-Exercise


In [10]:
y = train_label['label']

In [24]:
train_200 = pd.read_csv('train_aggs_200Times.csv', index_col = 'id')
test_200 = pd.read_csv('test_aggs_200Times.csv',index_col = 'id')

In [23]:
train_200

,acc_x_amax,acc_x_amin,acc_x_mean,acc_x_var,acc_x_median,acc_x_percentile_25,acc_x_percentile_75,acc_y_amax,acc_y_amin,acc_y_mean,acc_y_var,acc_y_median,acc_y_percentile_25,acc_y_percentile_75,acc_z_amax,acc_z_amin,acc_z_mean,acc_z_var,acc_z_median,acc_z_percentile_25,acc_z_percentile_75,gy_x_amax,gy_x_amin,gy_x_mean,gy_x_var,gy_x_median,gy_x_percentile_25,gy_x_percentile_75,gy_y_amax,gy_y_amin,gy_y_mean,gy_y_var,gy_y_median,gy_y_percentile_25,gy_y_percentile_75,gy_z_amax,gy_z_amin,gy_z_mean,gy_z_var,gy_z_median,gy_z_percentile_25,gy_z_percentile_75,acc_x_amax.1,acc_x_amin.1,acc_x_mean.1,acc_x_var.1,acc_x_median.1,acc_x_percentile_25.1,acc_x_percentile_75.1,acc_y_amax.1,acc_y_amin.1,acc_y_mean.1,acc_y_var.1,acc_y_median.1,acc_y_percentile_25.1,acc_y_percentile_75.1,acc_z_amax.1,acc_z_amin.1,acc_z_mean.1,acc_z_var.1,acc_z_median.1,acc_z_percentile_25.1,acc_z_percentile_75.1,gy_x_amax.1,gy_x_amin.1,gy_x_mean.1,gy_x_var.1,gy_x_median.1,gy_x_percentile_25.1,gy_x_percentile_75.1,gy_y_amax.1,gy_y_amin.1,gy_y_mean.1,gy_y_var.1,gy_y_median.1,gy_y_percentile_25.1,gy_y_percentile_75.1,gy_z_amax.1,gy_z_amin.1,gy_z_mean.1,gy_z_var.1,gy_z_median.1,gy_z_percentile_25.1,gy_z_percentile_75.1,acc_x_amax.2,acc_x_amin.2,acc_x_mean.2,acc_x_var.2,acc_x_median.2,acc_x_percentile_25.2,acc_x_percentile_75.2,acc_y_amax.2,acc_y_amin.2,acc_y_mean.2,acc_y_var.2,acc_y_median.2,acc_y_percentile_25.2,acc_y_percentile_75.2,acc_z_amax.2,acc_z_amin.2,acc_z_mean.2,acc_z_var.2,acc_z_median.2,acc_z_percentile_25.2,acc_z_percentile_75.2,gy_x_amax.2,gy_x_amin.2,gy_x_mean.2,gy_x_var.2,gy_x_median.2,gy_x_percentile_25.2,gy_x_percentile_75.2,gy_y_amax.2,gy_y_amin.2,gy_y_mean.2,gy_y_var.2,gy_y_median.2,gy_y_percentile_25.2,gy_y_percentile_75.2,gy_z_amax.2,gy_z_amin.2,gy_z_mean.2,gy_z_var.2,gy_z_median.2,gy_z_percentile_25.2,gy_z_percentile_75.2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,1.343113,0.613582,0.939629,0.040701,0.980503,0.736127,1.090305,0.133165,-0.624113,-0.240798,0.034805,-0.246550,-0.380588,-0.085563,-0.054876,-0.672738,-0.344088,0.021236,-0.316034,-0.413650,-0.236843,30.962266,-46.254836,-2.085009,146.879531,-1.653338,-9.013119,5.430400,69.847244,-85.887677,-2.103937,831.492026,-2.939646,-14.121287,10.771919,55.953827,-65.593383,5.986721,646.759919,4.805479,-5.325607,24.147286,1.284556,0.600915,0.926688,0.034010,0.954004,0.752065,1.068933,0.176871,-0.618396,-0.223190,0.033351,-0.259408,-0.369429,-0.061717,-0.124806,-0.754574,-0.361391,0.016644,-0.335702,-0.444566,-0.264865,31.644123,-44.233395,-2.094340,222.032780,-1.037531,-10.197549,8.286531,47.749768,-62.962850,-3.555653,554.276419,-2.445832,-16.042774,11.998392,53.111086,-79.930029,-1.571326,849.267181,-0.136255,-13.642551,21.437958,1.344268,0.591940,0.927670,0.035546,0.938673,0.751897,1.078447,0.161123,-0.452403,-0.191426,0.025026,-0.218429,-0.332050,-0.060161,-0.222396,-0.786336,-0.405785,0.015049,-0.381117,-0.478473,-0.313535,26.455876,-38.745614,-1.416459,161.968706,-1.207548,-10.215218,8.708907,38.686282,-54.420882,-4.418928,388.955182,-1.986291,-16.850110,5.023879,34.581731,-54.523817,-0.869073,391.849997,-1.113242,-10.858653,15.263659
1,-0.059166,-2.156208,-0.541579,0.256409,-0.256984,-0.877781,-0.150953,0.495337,-0.997057,-0.413170,0.149589,-0.584535,-0.741603,-0.077095,0.861091,-0.522313,0.313459,0.159479,0.448468,-0.060086,0.597465,176.742984,-322.482025,-3.292411,6398.842837,-14.009317,-42.463855,17.735028,221.240277,-315.096003,10.504189,6777.967302,5.955269,-5.534266,24.458588,93.343216,-179.514208,19.414557,1633.207053,16.395590,-6.370284,46.230189,0.316553,-2.090578,-0.924799,0.206331,-0.888976,-1.162463,-0.631008,0.228237,-1.295598,-0.424058,0.083410,-0.462348,-0.597532,-0.206750,0.682967,-0.923339,0.019893,0.103796,-0.004177,-0.189137,0.257300,286.624363,-325.328531,34.628383,8794.529016,43.879455,-6.624195,89.878353,389.608060,-306.613509,12.191602,10583.065294,17.276337,-37.669466,62.163876,340

`train_200`과 `test_200` 데이터셋은 aggFeatures2.ipynb에서 생성한 데이터셋을 csv로 저장하여 불러온 것이다.
id별로 `[np.max, np.min, np.mean, np.var, np.median, percentile(25), percentile(75)]`을 집계 계산한 결과이다.  

---

train_label을 확인해보면 총 label의 종류는 61가지, 그리고 그 중 운동이 아닌 ('Non-Exercise')가 가장 많다.  
그 다음으로 많은 'Device on Table'의 개수는 97개에 비해 Non-Exercise는 1518개로 레이블이 불균형하다는 것을 알 수 있다.    

--- 
이상 레이블을 검출하는 문제라고 생각했을 때, 이상 레이블을 가진 데이터의 수가 너무 적어 모델이 정상 레이블에 치우쳐진 학습이 이루어질 수 있다. 그로 인해 제대로된 이상 데이터 검출이 어려워지기 쉽다. 이를 해결하기 위해 Under Sampling 또는 Over Sampling 방법을 활용할 수 있다. Over Sampling 방식은 이상 데이터와 같이 적은 데이터 세트를 증식하여 학습을 위한 충분한 데이터를 확보하는 방법이다. `SMOTE`는 적은 데이터 세트에 있는 개별 데이터들의 K 최근접 이웃(K Nearest Neighbor)을 찾아 이 데이터와 K개의 이웃들의 차이를 일정 값으로 만들어서 기존 데이터와 약간 차이가 나는 새로운 데이터들을 생성하는 방식이다. 

---

Non-Exercise의 개수가 1518개고 나머지 labels들을 모두 1518개만큼 추가로 데이터를 생성해는 것이다. 그래서 총 SMOTE로 `fit_sample()`로 생성된 총 데이터 개수는 1518 * 61 = 92,598개. Over Sampling 하기 전 학습에 사용된 데이터의 크기는 3125개로 적은 개수였다. 그러나 Over Sampling 로 추가 생성되어 학습에 필요한 데이터을 가지고 있음으로 그동안 사용했던 XGBoost 모델과 더불어 LightGBM 모델을 사용해도 가능할 것으로 보인다.  

In [26]:
from imblearn.over_sampling import SMOTE

In [27]:
# SMOTE oversampling

smote = SMOTE(random_state = 0 )

In [28]:
X_train_over, y_train_over = smote.fit_sample(train_200, y)

In [29]:
print('SMOTE 적용 전 학습용 feature/label 데이터 세트 : ', train_200.shape, y.shape)
print('SMOTE 적용 후 학습용 feature/label 데이터 세트 : ', X_train_over.shape, y_train_over.shape)

SMOTE 적용 전 학습용 feature/label 데이터 세트 :  (3125, 126) (3125,)
SMOTE 적용 후 학습용 feature/label 데이터 세트 :  (92598, 126) (92598,)


In [30]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X_train_over, y_train_over, test_size = 0.2, 
                                                   random_state = 42)
print(y_train.nunique(), y_test.nunique())

61 61


In [33]:
xgb = XGBClassifier(n_estimators = 400, learning_rate = 0.05, 
                    max_depth = 6, 
                    objective = 'multi:softprob', random_state = 0) 

In [37]:
evals = [(X_test.values, y_test)]

In [39]:
X_test

,acc_x_amax,acc_x_amin,acc_x_mean,acc_x_var,acc_x_median,acc_x_percentile_25,acc_x_percentile_75,acc_y_amax,acc_y_amin,acc_y_mean,acc_y_var,acc_y_median,acc_y_percentile_25,acc_y_percentile_75,acc_z_amax,acc_z_amin,acc_z_mean,acc_z_var,acc_z_median,acc_z_percentile_25,acc_z_percentile_75,gy_x_amax,gy_x_amin,gy_x_mean,gy_x_var,gy_x_median,gy_x_percentile_25,gy_x_percentile_75,gy_y_amax,gy_y_amin,gy_y_mean,gy_y_var,gy_y_median,gy_y_percentile_25,gy_y_percentile_75,gy_z_amax,gy_z_amin,gy_z_mean,gy_z_var,gy_z_median,gy_z_percentile_25,gy_z_percentile_75,acc_x_amax.1,acc_x_amin.1,acc_x_mean.1,acc_x_var.1,acc_x_median.1,acc_x_percentile_25.1,acc_x_percentile_75.1,acc_y_amax.1,acc_y_amin.1,acc_y_mean.1,acc_y_var.1,acc_y_median.1,acc_y_percentile_25.1,acc_y_percentile_75.1,acc_z_amax.1,acc_z_amin.1,acc_z_mean.1,acc_z_var.1,acc_z_median.1,acc_z_percentile_25.1,acc_z_percentile_75.1,gy_x_amax.1,gy_x_amin.1,gy_x_mean.1,gy_x_var.1,gy_x_median.1,gy_x_percentile_25.1,gy_x_percentile_75.1,gy_y_amax.1,gy_y_amin.1,gy_y_mean.1,gy_y_var.1,gy_y_median.1,gy_y_percentile_25.1,gy_y_percentile_75.1,gy_z_amax.1,gy_z_amin.1,gy_z_mean.1,gy_z_var.1,gy_z_median.1,gy_z_percentile_25.1,gy_z_percentile_75.1,acc_x_amax.2,acc_x_amin.2,acc_x_mean.2,acc_x_var.2,acc_x_median.2,acc_x_percentile_25.2,acc_x_percentile_75.2,acc_y_amax.2,acc_y_amin.2,acc_y_mean.2,acc_y_var.2,acc_y_median.2,acc_y_percentile_25.2,acc_y_percentile_75.2,acc_z_amax.2,acc_z_amin.2,acc_z_mean.2,acc_z_var.2,acc_z_median.2,acc_z_percentile_25.2,acc_z_percentile_75.2,gy_x_amax.2,gy_x_amin.2,gy_x_mean.2,gy_x_var.2,gy_x_median.2,gy_x_percentile_25.2,gy_x_percentile_75.2,gy_y_amax.2,gy_y_amin.2,gy_y_mean.2,gy_y_var.2,gy_y_median.2,gy_y_percentile_25.2,gy_y_percentile_75.2,gy_z_amax.2,gy_z_amin.2,gy_z_mean.2,gy_z_var.2,gy_z_median.2,gy_z_percentile_25.2,gy_z_percentile_75.2
20899,-0.475059,-1.580237,-0.997151,0.106090,-1.057743,-1.276677,-0.725587,0.509108,-0.484729,0.002880,0.036382,0.015780,-0.139783,0.142720,0.321403,-1.210364,-0.135338,0.080662,-0.095962,-0.318168,0.081474,93.896921,-156.939914,-3.479743,2263.802382,2.735807,-29.557270,31.320993,104.659098,-106.696264,3.139892,2244.555869,2.295932,-34.066049,37.883339,82.046497,-70.142986,0.381037,1413.806832,-6.005020,-22.074972,19.859401,-0.505019,-1.480200,-1.033808,0.079900,-1.113033,-1.250424,-0.804400,0.441158,-0.471546,0.026978,0.030405,0.040441,-0.090406,0.150445,0.340982,-0.575483,-0.139328,0.050840,-0.131761,-0.331159,0.029084,111.159814,-136.839391,-2.731560,1742.527904,-2.523164,-27.050045,28.813398,67.718554,-81.229800,0.078548,1558.568657,-2.568590,-31.057512,34.331057,63.774180,-93.627082,-6.210977,986.321601,-7.559090,-24.850558,12.602723,-0.535542,-1.466061,-0.987948,0.078187,-0.945733,-1.222379,-0.771291,0.440257,-0.424392,0.035388,0.031968,0.040538,-0.083984,0.146894,0.275242,-0.654603,-0.169662,0.050991,-0.177781,-0.351303,0.017100,72.324059,-110.263525,-2.548064,1400.023862,-0.729596,-21.675540,24.235220,74.205899,-84.199278,2.913084,1587.657300,3.722784,-25.156304,34.686550,81.968925,-67.279136,-0.957133,1102.656802,-5.392985,-21.477285,16.012342
65414,0.075172,-1.046176,-0.453056,0.072646,-0.415666,-0.630809,-0.252494,-0.258833,-1.475329,-0.831582,0.108327,-0.819853,-1.051015,-0.524855,0.691279,-0.218095,0.142839,0.030677,0.125889,0.018855,0.251726,87.455445,-85.611400,-4.801409,1652.639038,-3.411445,-36.176826,21.804240,87.139050,-111.320534,-10.612775,1983.384308,-7.172031,-42.634625,13.360375,118.695653,-170.495646,-9.987321,5205.611373,-5.584922,-61.962817,45.108796,0.041488,-1.186284,-0.465706,0.098810,-0.458584,-0.632591,-0.203467,-0.165497,-1.615532,-0.751806,0.132966,-0.651900,-0.989330,-0.489934,0.602342,-0.282230,0.126426,0.048077,0.109181,-0.056845,0.292250,115.014414,-104.120401,-4.663000,2752.186739,-2.698255,-46.528873,33.797645,121.145903,-128.495599,0.038853,2585.247196,2.097727,-27.725984,22.282155,175.137258,-196.059990,-5.858395,9461.853362,-7.451948,-74.405644,52.035096,0.098814,-1.060410,-0.442844,0.0

In [40]:
start = time.time()
xgb.fit(X_train_over.values, y_train_over,
            eval_metric = 'mlogloss', 
            eval_set= evals, 
            early_stopping_rounds = 100)

sec = time.time()-start
times = str(datetime.timedelta(seconds = sec)).split('.')

print('소요시간 : ', times)

[0]	validation_0-mlogloss:2.90889
[1]	validation_0-mlogloss:2.53030
[2]	validation_0-mlogloss:2.26883
[3]	validation_0-mlogloss:2.06737
[4]	validation_0-mlogloss:1.90158
[5]	validation_0-mlogloss:1.76174
[6]	validation_0-mlogloss:1.63911
[7]	validation_0-mlogloss:1.53177
[8]	validation_0-mlogloss:1.43617
[9]	validation_0-mlogloss:1.34949
[10]	validation_0-mlogloss:1.27109
[11]	validation_0-mlogloss:1.19940
[12]	validation_0-mlogloss:1.13359
[13]	validation_0-mlogloss:1.07303
[14]	validation_0-mlogloss:1.01684
[15]	validation_0-mlogloss:0.96458
[16]	validation_0-mlogloss:0.91582
[17]	validation_0-mlogloss:0.87004
[18]	validation_0-mlogloss:0.82739
[19]	validation_0-mlogloss:0.78721
[20]	validation_0-mlogloss:0.74932
[21]	validation_0-mlogloss:0.71379
[22]	validation_0-mlogloss:0.68004
[23]	validation_0-mlogloss:0.64814
[24]	validation_0-mlogloss:0.61802
[25]	validation_0-mlogloss:0.58962
[26]	validation_0-mlogloss:0.56249
[27]	validation_0-mlogloss:0.53680
[28]	validation_0-mlogloss:0.5

In [41]:
y_pred = xgb.predict_proba(test_200.values)
submission.iloc[:,1:] = y_pred
submission.to_csv('submission파일/over_xgb200T.csv', index =False)

제출 결과값 : 0.834271837   
Over Sampling을 하지 않은 데이터셋으로 학습한 모델이 더 좋은 결과값을 가짐. 

In [42]:
# LightGBM
from lightgbm import LGBMClassifier
lgbm = LGBMClassifier(n_estimators = 400, learning_rate = 0.05, random_state = 0)
start = time.time()
lgbm.fit(X_train_over, y_train_over, eval_set = evals,
        early_stopping_rounds = 100)
sec = time.time() -start 

print('수행시간 : ', str(datetime.timedelta(seconds = sec)).split('.'))

[1]	valid_0's multi_logloss: 1.57191
Training until validation scores don't improve for 100 rounds
[2]	valid_0's multi_logloss: 1.34233
[3]	valid_0's multi_logloss: 1.17811
[4]	valid_0's multi_logloss: 1.04513
[5]	valid_0's multi_logloss: 0.931983
[6]	valid_0's multi_logloss: 0.835131
[7]	valid_0's multi_logloss: 0.750146
[8]	valid_0's multi_logloss: 0.676082
[9]	valid_0's multi_logloss: 0.610327
[10]	valid_0's multi_logloss: 0.551632
[11]	valid_0's multi_logloss: 0.499288
[12]	valid_0's multi_logloss: 0.452491
[13]	valid_0's multi_logloss: 0.410426
[14]	valid_0's multi_logloss: 0.372459
[15]	valid_0's multi_logloss: 0.33831
[16]	valid_0's multi_logloss: 0.307418
[17]	valid_0's multi_logloss: 0.279412
[18]	valid_0's multi_logloss: 0.254094
[19]	valid_0's multi_logloss: 0.231141
[20]	valid_0's multi_logloss: 0.210416
[21]	valid_0's multi_logloss: 0.191407
[22]	valid_0's multi_logloss: 0.174318
[23]	valid_0's multi_logloss: 0.158783
[24]	valid_0's multi_logloss: 0.144657
[25]	valid_0's m

In [43]:
y_pred = lgbm.predict_proba(test_200.values)
submission.iloc[:,1:] = y_pred
submission.to_csv('submission파일/over_lgbm200T.csv', index = False)

제출결과값 : 1.1567946727

In [45]:
def percentile(n):
    def percentile_(x):
        return np.percentile(x, n)
    percentile_.__name__ = 'percentile_%s' % n
    return percentile_

In [47]:
id_train = train.drop('time',axis = 1).groupby('id').agg([np.max, np.min, np.mean, 
                       np.var, np.median,
                      percentile(25), percentile(75)])

In [49]:
smote = SMOTE(random_state = 0 )
X_over, y_over = smote.fit_sample(id_train, y)

In [50]:
X_over.shape

(92598, 42)

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size = 0.2,
                                                    random_state = 42)
evals = [(X_test.values, y_test)]

In [54]:
start = time.time()
xgb.fit(X_over.values, y_over, 
       eval_metric = 'mlogloss',
       eval_set = evals,
       early_stopping_rounds = 100)
sec = time.time()-start
print('소요시간 : ', str(datetime.timedelta(seconds = sec)).split('.'))

[0]	validation_0-mlogloss:2.94802
[1]	validation_0-mlogloss:2.56972
[2]	validation_0-mlogloss:2.30743
[3]	validation_0-mlogloss:2.10514
[4]	validation_0-mlogloss:1.93954
[5]	validation_0-mlogloss:1.79955
[6]	validation_0-mlogloss:1.67840
[7]	validation_0-mlogloss:1.57161
[8]	validation_0-mlogloss:1.47642
[9]	validation_0-mlogloss:1.39064
[10]	validation_0-mlogloss:1.31301
[11]	validation_0-mlogloss:1.24189
[12]	validation_0-mlogloss:1.17677
[13]	validation_0-mlogloss:1.11616
[14]	validation_0-mlogloss:1.05996
[15]	validation_0-mlogloss:1.00773
[16]	validation_0-mlogloss:0.95941
[17]	validation_0-mlogloss:0.91392
[18]	validation_0-mlogloss:0.87130
[19]	validation_0-mlogloss:0.83125
[20]	validation_0-mlogloss:0.79312
[21]	validation_0-mlogloss:0.75742
[22]	validation_0-mlogloss:0.72374
[23]	validation_0-mlogloss:0.69153
[24]	validation_0-mlogloss:0.66132
[25]	validation_0-mlogloss:0.63251
[26]	validation_0-mlogloss:0.60513
[27]	validation_0-mlogloss:0.57919
[28]	validation_0-mlogloss:0.5

In [55]:
id_test = test.drop('time',axis = 1).groupby('id').agg([np.max, np.min, np.mean, 
                       np.var, np.median,
                      percentile(25), percentile(75)])

In [56]:
y_pred = xgb.predict_proba(id_test.values)
submission.iloc[:,1:]=y_pred
submission.to_csv('submission파일/over_xgb.csv',index =False)

제출 결과값 : 1.0243872569